## import needed libraries

In [17]:
import joblib
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import numpy as np
import os

## Create a validation (simulation) dataset - get last N bars

In [18]:
def full_data(symbol = "AMD", exchange = "NASDAQ"):
    interval = Interval.in_1_minute
    n_bars = 5000
    tv = TvDatafeed()

    df = tv.get_hist(symbol=symbol, exchange=exchange, interval=interval, n_bars=n_bars)
    df = df.drop(columns=['symbol'])

    def compute_rsi(series, window=14):
        delta = series.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)

        avg_gain = gain.rolling(window=window, min_periods=1).mean()
        avg_loss = loss.rolling(window=window, min_periods=1).mean()

        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    def compute_macd(series, short=12, long=26, signal=9):
        ema_short = series.ewm(span=short, adjust=False).mean()
        ema_long = series.ewm(span=long, adjust=False).mean()
        macd_line = ema_short - ema_long
        signal_line = macd_line.ewm(span=signal, adjust=False).mean()
        return macd_line, signal_line

    def compute_bollinger_bands(series, window=20, num_std=2):
        sma = series.rolling(window=window).mean()
        std_dev = series.rolling(window=window).std()
        upper_band = sma + (num_std * std_dev)
        lower_band = sma - (num_std * std_dev)
        return sma, upper_band, lower_band

    def compute_features(df):
        # Compute RSI
        df['RSI'] = compute_rsi(df['close'])
        df['RSI_Change'] = df['RSI'].diff()

        # Compute Bollinger Bands
        df['SMA_20'], df['Bollinger_Upper'], df['Bollinger_Lower'] = compute_bollinger_bands(df['close'])
        df['SMA_10'] = df['close'].rolling(10).mean()
        df['SMA_5'] = df['close'].rolling(5).mean()

        # Feature Engineering for SMA
        df['SMA_5_Distance'] = (df['SMA_5'] - df['close']) / df['close']
        df['SMA_10_Distance'] = (df['SMA_10'] - df['close']) / df['close']
        df['SMA_Crossover'] = (df['SMA_5'] - df['SMA_10']) / df['close']

        # Calculate Volume features
        df['Volume_MA'] = df['volume'].rolling(window=50).mean()
        df['Volume_Spike'] = df['volume'] / df['Volume_MA']
        df['MACD'] = df['close'].ewm(span=12).mean() - df['close'].ewm(span=26).mean()

        # Normalized MACD
        df['Normalized_MACD'] = df['MACD'] / (df['MACD'].ewm(span=9, adjust=False).mean())

        # Normalized Bollinger Bands
        df['Normalized_BB'] = (df['close'] - df['Bollinger_Lower']) / (df['Bollinger_Upper'] - df['Bollinger_Lower'])

        # Rolling volatility (standard deviation of returns)
        df['Volatility_10_Days'] = df['close'].pct_change().rolling(window=10).std() * 100
        df['Volatility_20_Days'] = df['close'].pct_change().rolling(window=20).std() * 100

        # Target Value
        df['Mean_10_Days_Ahead'] = df['close'].shift(-10).rolling(window=10, min_periods=1).mean()
        df['Target'] = (df['Mean_10_Days_Ahead'] - df['close']) / df['close'] * 100

        # Drop raw unbounded features
        df = df.drop(columns=['SMA_10', 'SMA_5'])

        return df


    df = compute_features(df)
    df = df.dropna()

    return df


## I'm assuming we have modelstrained models' parameters in analysis/models, getting prediction:

In [19]:
def single_shot(data):
    """
        gets n bars from a given time and predicts the movements
    """

    wanted_features = ['SMA_Crossover', 'RSI' ,'Volatility_10_Days', 'Volatility_20_Days'] 
    data = data[wanted_features]

    # Select the last n time_steps (for LSTM input)
    time_steps = 5
    if len(data) < time_steps:
        raise ValueError("Not enough data to create the required time steps for LSTM input.")

    # Extract last `time_steps` rows and reshape for LSTM input
    lstm_data = data.iloc[-time_steps:].to_numpy().reshape(1, time_steps, -1)

    # Extract the last row for standard models
    standard_data = data.iloc[-1].to_numpy().reshape(1, -1)
    print(lstm_data)
    return standard_data, lstm_data


def analysis_predict(data):
    """
    Load models and calculate the average prediction based on the data.
    """
    standard_data, lstm_data = single_shot(data)

    models_dir = "../analisys/models"
    model_files = [os.path.join(models_dir, file) for file in os.listdir(models_dir) if file.endswith(".pkl")]

    predictions = []
    for file_path in model_files:
        model = joblib.load(file_path)
        if "lstm" in file_path.lower():
            lstm_prediction = model.predict(lstm_data)
            predictions.append(lstm_prediction.flatten()[0])
        else:
            predictions.append(model.predict(standard_data)[0])  

    # normalize to [0, 1]
    # print(predictions)
    pred = np.mean(predictions)
    pred = (pred + 3) / 6
    print(pred)
    return pred


## now lets create some strategies using our bots

### strategy to manually manage all trades

In [20]:
def strategy_manual(data, state):
    '''
    manually open/close all trades
    '''
    balance, stock_worth = state
    pred = analysis_predict(data)
    if pred > 0.58:
        amount = 0.10 * balance
        return {'act': 'buy', 'amount': amount}
    elif pred < 0.515:
        amount = 0.5 * stock_worth
        return {'act': 'sell', 'amount': amount}
    else:
        return {'act': 'hold'}

def simulate_manual(full_data, strategy):
    '''
    gets a full data set and a strategy, buys and sells [amount] based on a strategy
    '''
    starting_bal = 100000
    balance = starting_bal
    shares = 0.0
    n_bars = len(full_data)
    curr_bar = 5
    buy_actions = 0
    sell_actions = 0

    while curr_bar < n_bars:
        shot_data = full_data.iloc[curr_bar-5 : curr_bar]
        shot_data = shot_data.drop(columns=['open', 'close', 'high', 'low'])
        current_price = full_data.iloc[curr_bar]['close']
        current_stock_worth = shares * current_price

        action = strategy(shot_data, (balance, current_stock_worth))

        if action['act'] == 'buy':
            amount = action.get('amount', 0)
            amount = min(amount, balance)
            if amount > 0 and current_price > 0:
                shares_bought = amount / current_price
                balance -= amount
                shares += shares_bought
                buy_actions += 1
        elif action['act'] == 'sell':
            amount = action.get('amount', 0)
            if amount > 0 and current_price > 0:
                shares_to_sell = amount / current_price
                shares_to_sell = min(shares_to_sell, shares)
                balance += shares_to_sell * current_price
                shares -= shares_to_sell
                sell_actions += 1

        curr_bar += 1

    # Sell all remaining shares at the last close price
    if shares > 0:
        final_price = full_data.iloc[-1]['close']
        balance += shares * final_price
        sell_actions += 1
        shares = 0

    print(f"Buys: {buy_actions}, sells: {sell_actions}")

    total_return = balance / starting_bal
    return total_return

In [29]:
symbol = "AMD"
exchange = "NASDAQ"
simul_len = 15

# LEFT LOW SIMULATION LENGTH TO KEEP THINGS CLEAN

df = full_data(symbol, exchange)[0:simul_len]
strat = strategy_manual
res = simulate_manual(df, strat)
print(f"total return: {res - 1.0:.3f}%")

[[[ 1.04667390e-03  6.21951220e+01  1.28782734e-01  1.03984702e-01]
  [ 5.98383587e-04  4.94505495e+01  1.42039187e-01  1.17764595e-01]
  [ 6.99409388e-05  4.55621302e+01  1.39872717e-01  1.13938227e-01]
  [-4.89168414e-04  5.14619883e+01  1.37734749e-01  1.13593183e-01]
  [-1.39191291e-03  4.51612903e+01  1.46319090e-01  1.17369515e-01]]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
0.5189005470687987
[[[ 5.98383587e-04  4.94505495e+01  1.42039187e-01  1.17764595e-01]
  [ 6.99409388e-05  4.55621302e+01  1.39872717e-01  1.13938227e-01]
  [-4.89168414e-04  5.14619883e+01  1.37734749e-01  1.13593183e-01]
  [-1.39191291e-03  4.51612903e+01  1.46319090e-01  1.17369515e-01]
  [-1.86974135e-03  3.43750000e+01  1.27350123e-01  1.24289334e-01]]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
0.5119638082237504
[[[ 6.99409388e-05  4.55621302e+01  1.39872717e-01  1.13938227e-01]
  [-4.89168414e-04  5.14619883e+01  1.37734749e-01  1.13593183e-01]
  [-1.39191291e-03  4.51612903e+01  1.46319090e-01  1.17369515e-

### strategy for only opening trades, with set tp and sl

In [30]:
def strategy_tpsl(data, state):
    """
    Strategy that opens trades with set stop loss and take profit.
    """
    balance, stock_worth = state
    pred = analysis_predict(data)
    # Buy condition with SL=1%, TP=2%
    if pred > 0.58:
        amount = 0.10 * balance
        return {
            'act': 'buy', 
            'amount': amount,
            'stop_loss': 0.01,
            'take_profit': 0.02
        }
    else:
        return {'act': 'hold'}

def simulate_tpsl(full_data, strategy):
    """
    Simulates trading with stop loss and take profit management.
    """
    starting_bal = 100000
    balance = starting_bal
    open_trades = []
    n_bars = len(full_data)
    curr_bar = 5
    buy_actions = 0
    total_trades_closed = 0

    while curr_bar < n_bars:
        current_bar_data = full_data.iloc[curr_bar]
        current_price = current_bar_data['close']
        current_high = current_bar_data['high']
        current_low = current_bar_data['low']

        for trade in open_trades.copy():
            tp_price = trade['take_profit']
            sl_price = trade['stop_loss']
            
            if current_high >= tp_price:
                balance += trade['shares'] * tp_price
                open_trades.remove(trade)
                total_trades_closed += 1
            elif current_low <= sl_price:
                balance += trade['shares'] * sl_price
                open_trades.remove(trade)
                total_trades_closed += 1

        shot_data = full_data.iloc[curr_bar-5:curr_bar].drop(
            columns=['open', 'close', 'high', 'low']
        )
        current_stock_worth = sum(t['shares'] * current_price for t in open_trades)
        action = strategy(shot_data, (balance, current_stock_worth))

        if action['act'] == 'buy':
            amount = min(action['amount'], balance)
            if amount > 0 and current_price > 0:
                shares = amount / current_price
                sl = current_price * (1 - action['stop_loss'])
                tp = current_price * (1 + action['take_profit'])
                
                open_trades.append({
                    'entry_price': current_price,
                    'stop_loss': sl,
                    'take_profit': tp,
                    'shares': shares
                })
                balance -= amount
                buy_actions += 1

        curr_bar += 1

    # Close remaining positions at final price
    final_price = full_data.iloc[-1]['close']
    for trade in open_trades:
        balance += trade['shares'] * final_price
    total_trades_closed += len(open_trades)

    print(f"Total Trades Opened: {buy_actions}")
    print(f"Total Trades Closed: {total_trades_closed}")
    
    return balance / starting_bal

In [31]:
symbol = "AMD"
exchange = "NASDAQ"
simul_len = 15

# LEFT LOW SIMULATION LENGTH TO KEEP THINGS CLEAN

df = full_data(symbol, exchange)[0:simul_len]
strat = strategy_tpsl
res = simulate_tpsl(df, strat)
print(f"Final Return: {(res - 1)*100:.2f}%")

[[[ 1.04667390e-03  6.21951220e+01  1.28782734e-01  1.03984702e-01]
  [ 5.98383587e-04  4.94505495e+01  1.42039187e-01  1.17764595e-01]
  [ 6.99409388e-05  4.55621302e+01  1.39872717e-01  1.13938227e-01]
  [-4.89168414e-04  5.14619883e+01  1.37734749e-01  1.13593183e-01]
  [-1.39191291e-03  4.51612903e+01  1.46319090e-01  1.17369515e-01]]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
0.5189005470687987
[[[ 5.98383587e-04  4.94505495e+01  1.42039187e-01  1.17764595e-01]
  [ 6.99409388e-05  4.55621302e+01  1.39872717e-01  1.13938227e-01]
  [-4.89168414e-04  5.14619883e+01  1.37734749e-01  1.13593183e-01]
  [-1.39191291e-03  4.51612903e+01  1.46319090e-01  1.17369515e-01]
  [-1.86974135e-03  3.43750000e+01  1.27350123e-01  1.24289334e-01]]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
0.5119638082237504
[[[ 6.99409388e-05  4.55621302e+01  1.39872717e-01  1.13938227e-01]
  [-4.89168414e-04  5.14619883e+01  1.37734749e-01  1.13593183e-01]
  [-1.39191291e-03  4.51612903e+01  1.46319090e-01  1.17369515e-

### only opening trades, with a trailing stop loss and two take profit targets

In [32]:
def strategy_trailingsl(data, state):
    """
    Strategy that opens trades with two TP targets and breakeven SL adjustment
    """
    balance, stock_worth = state
    pred = analysis_predict(data)
    
    if pred > 0.58:
        amount = 0.10 * balance
        return {
            'act': 'buy', 
            'amount': amount,
            'stop_loss': 0.01,
            'take_profit1': 0.01,
            'take_profit2': 0.02
        }
    else:
        return {'act': 'hold'}

def simulate_trailingsl(full_data, strategy):
    """
    Simulates trading with two TP targets and breakeven adjustment
    """
    starting_bal = 100000
    balance = starting_bal
    open_trades = []
    n_bars = len(full_data)
    curr_bar = 5
    buy_actions = 0
    total_trades_closed = 0

    while curr_bar < n_bars:
        current_bar_data = full_data.iloc[curr_bar]
        current_price = current_bar_data['close']
        current_high = current_bar_data['high']
        current_low = current_bar_data['low']

        for trade in open_trades.copy():
            if trade['stage'] == 1:
                tp = trade['take_profit1']
                sl = trade['stop_loss']
            else:
                tp = trade['take_profit2']
                sl = trade['breakeven_sl']

            if current_high >= tp:
                if trade['stage'] == 1:
                    # Close 50% position at TP1 and adjust SL to breakeven
                    close_shares = trade['shares'] * 0.5
                    balance += close_shares * tp
                    trade['shares'] -= close_shares
                    trade['stage'] = 2
                    trade['breakeven_sl'] = trade['entry_price']
                    total_trades_closed += 0.5
                else:
                    balance += trade['shares'] * tp
                    open_trades.remove(trade)
                    total_trades_closed += 1

            elif current_low <= sl:
                balance += trade['shares'] * sl
                open_trades.remove(trade)
                total_trades_closed += 1 if trade['stage'] == 1 else 0.5

        shot_data = full_data.iloc[curr_bar-5:curr_bar].drop(
            columns=['open', 'close', 'high', 'low']
        )
        current_stock_worth = sum(t['shares'] * current_price for t in open_trades)
        action = strategy(shot_data, (balance, current_stock_worth))

        if action['act'] == 'buy':
            amount = min(action['amount'], balance)
            if amount > 0 and current_price > 0:
                shares = amount / current_price
                initial_sl = current_price * (1 - action['stop_loss'])
                tp1 = current_price * (1 + action['take_profit1'])
                tp2 = current_price * (1 + action['take_profit2'])
                
                open_trades.append({
                    'entry_price': current_price,
                    'stop_loss': initial_sl,
                    'breakeven_sl': None,
                    'take_profit1': tp1,
                    'take_profit2': tp2,
                    'shares': shares,
                    'stage': 1
                })
                balance -= amount
                buy_actions += 1

        curr_bar += 1

    final_price = full_data.iloc[-1]['close']
    for trade in open_trades:
        balance += trade['shares'] * final_price
    total_trades_closed += len(open_trades)

    print(f"Total Trades Opened: {buy_actions}")
    print(f"Total Trades Closed: {round(total_trades_closed, 1)}")
    
    return balance / starting_bal

In [33]:
symbol = "AMD"
exchange = "NASDAQ"
simul_len = 15

# LEFT LOW SIMULATION LENGTH TO KEEP THINGS CLEAN

df = full_data(symbol, exchange)[0:simul_len]
strat = strategy_trailingsl
res = simulate_trailingsl(df, strat)
print(f"Final Return: {(res - 1)*100:.2f}%")

[[[ 1.04667390e-03  6.21951220e+01  1.28782734e-01  1.03984702e-01]
  [ 5.98383587e-04  4.94505495e+01  1.42039187e-01  1.17764595e-01]
  [ 6.99409388e-05  4.55621302e+01  1.39872717e-01  1.13938227e-01]
  [-4.89168414e-04  5.14619883e+01  1.37734749e-01  1.13593183e-01]
  [-1.39191291e-03  4.51612903e+01  1.46319090e-01  1.17369515e-01]]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
0.5189005470687987
[[[ 5.98383587e-04  4.94505495e+01  1.42039187e-01  1.17764595e-01]
  [ 6.99409388e-05  4.55621302e+01  1.39872717e-01  1.13938227e-01]
  [-4.89168414e-04  5.14619883e+01  1.37734749e-01  1.13593183e-01]
  [-1.39191291e-03  4.51612903e+01  1.46319090e-01  1.17369515e-01]
  [-1.86974135e-03  3.43750000e+01  1.27350123e-01  1.24289334e-01]]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
0.5119638082237504
[[[ 6.99409388e-05  4.55621302e+01  1.39872717e-01  1.13938227e-01]
  [-4.89168414e-04  5.14619883e+01  1.37734749e-01  1.13593183e-01]
  [-1.39191291e-03  4.51612903e+01  1.46319090e-01  1.17369515e-

## That was tested on the same symbol as the models were teached on but with a different timeframes, let's see how the bots perform on a different symbol, let's say AAPL

### Results:
for manual: (return, opened trades)  
for other: (return, opened trades, tp hits, sl hits)

amd manual  
(np.float64(0.927376450003768), 279)  
amd tpsl  
(np.float64(1.0034570022025326), 279, 69, 187)  
amd trailing  
(np.float64(1.008171219426555), 304.0, 182, 207)  

apple manual  
(np.float64(0.973602225864892), 335)  
apple tpsl  
(np.float64(0.9953122580888374), 335, 89, 181)  
apple trailing  
(np.float64(0.994153136797656), 375.5, 179, 213)


### prettier data is on slides